# Bert text classification on SageMaker using PyTorch

This uses the [dbpedia dataset](https://wiki.dbpedia.org/services-resources/dbpedia-data-set-2014#2) and BERT for text classification. The dataset csv looks like

```text
1,"E. D. Abbott Ltd"," Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972."
1,"Schwan-Stabilo"," Schwan-STABILO is a German maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. It is the world's largest manufacturer of highlighter pens Stabilo Boss."
1,"Q-workshop"," Q-workshop is a Polish company located in Poznań that specializes in designand production of polyhedral dice and dice accessories for use in various games (role-playing gamesboard games and tabletop wargames). They also run an online retail store and maintainan active forum community.Q-workshop was established in 2001 by Patryk Strzelewicz – a student from Poznań. Initiallythe company sold its products via online auction services but in 2005 a website and online store wereestablished."
```



In [1]:
!pip install -r requirements_notebook.txt

In [2]:
import sys, os
import logging

sys.path.append("src")

logging.basicConfig(level="INFO", handlers=[logging.StreamHandler(sys.stdout)],
                        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

### Bucket and role set up

In [6]:
import sagemaker
from sagemaker import get_execution_role
sm_session = sagemaker.session.Session()
role = 'SageMaker_admin'

2021-03-03 12:43:08,423 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [7]:
data_bucket = sm_session.default_bucket()

data_bucket_prefix = "bert-demo"

s3_uri_data = "s3://{}/{}/data".format(data_bucket, data_bucket_prefix)
s3_uri_train = "{}/{}".format(s3_uri_data, "train.csv")
s3_uri_val = "{}/{}".format(s3_uri_data, "val.csv")

# Use a small data set if required for local testing..
s3_uri_mini_data = "s3://{}/{}/minidata".format(data_bucket, data_bucket_prefix)
s3_uri_mini_train = "{}/{}".format(s3_uri_mini_data, "train.csv")
s3_uri_mini_val = "{}/{}".format(s3_uri_mini_data, "val.csv")

s3_uri_classes = "{}/{}".format(s3_uri_data, "classes.txt")

s3_uri_test = "{}/{}".format(s3_uri_data, "test.csv")

s3_output_path = "s3://{}/{}/output".format(data_bucket, data_bucket_prefix)
s3_code_path = "s3://{}/{}/code".format(data_bucket, data_bucket_prefix)
s3_checkpoint = "s3://{}/{}/checkpoint".format(data_bucket, data_bucket_prefix)

2021-03-03 12:43:13,083 - sagemaker - INFO - Created S3 bucket: sagemaker-ca-central-1-114590703656


In [8]:
prepare_dataset = True

## Prepare dataset

In [9]:
tmp ="tmp"

In [10]:
%%bash -s  "$prepare_dataset"  "$s3_uri_test" "$s3_uri_classes" "$tmp"
   
prepare_dataset=$1
s3_test=$2
s3_classes=$3
tmp=$4

if [ "$prepare_dataset" == "True" ]
then  
    echo "Downloading data.."
    wget https://github.com/saurabh3949/Text-Classification-Datasets/raw/master/dbpedia_csv.tar.gz -P ${tmp}
    tar -xzvf ${tmp}/dbpedia_csv.tar.gz
    mv dbpedia_csv ${tmp}
    
    ls -l ${tmp}/dbpedia_csv/
    cat  ${tmp}/dbpedia_csv/classes.txt
    head -3 ${tmp}/dbpedia_csv/train.csv 
    
    echo aws s3 cp ${tmp}/dbpedia_csv/test.csv ${s3_test}
    aws s3 cp ${tmp}/dbpedia_csv/test.csv ${s3_test}
    
    aws s3 cp ${tmp}/dbpedia_csv/classes.txt ${s3_classes}
   
fi

aws s3 cp tmp/dbpedia_csv/test.csv s3://sagemaker-ca-central-1-114590703656/bert-demo/data/test.csv
bash: line 10: wget: command not found
tar: Error opening archive: Failed to open 'tmp/dbpedia_csv.tar.gz'
mv: rename dbpedia_csv to tmp: No such file or directory
ls: tmp/dbpedia_csv/: No such file or directory
cat: tmp/dbpedia_csv/classes.txt: No such file or directory
head: tmp/dbpedia_csv/train.csv: No such file or directory

The user-provided path tmp/dbpedia_csv/test.csv does not exist.

The user-provided path tmp/dbpedia_csv/classes.txt does not exist.


CalledProcessError: Command 'b'   \nprepare_dataset=$1\ns3_test=$2\ns3_classes=$3\ntmp=$4\n\nif [ "$prepare_dataset" == "True" ]\nthen  \n    echo "Downloading data.."\n    wget https://github.com/saurabh3949/Text-Classification-Datasets/raw/master/dbpedia_csv.tar.gz -P ${tmp}\n    tar -xzvf ${tmp}/dbpedia_csv.tar.gz\n    mv dbpedia_csv ${tmp}\n    \n    ls -l ${tmp}/dbpedia_csv/\n    cat  ${tmp}/dbpedia_csv/classes.txt\n    head -3 ${tmp}/dbpedia_csv/train.csv \n    \n    echo aws s3 cp ${tmp}/dbpedia_csv/test.csv ${s3_test}\n    aws s3 cp ${tmp}/dbpedia_csv/test.csv ${s3_test}\n    \n    aws s3 cp ${tmp}/dbpedia_csv/classes.txt ${s3_classes}\n   \nfi\n'' returned non-zero exit status 255.

#### Train val split

In [ ]:
from sklearn.model_selection import train_test_split

def train_val_split(data_file, train_file_name = None, val_file_name = None, val_ratio =.30, train_ratio = .70):
    with open(data_file, "r") as f:
        lines = f.readlines()
        
    train, val = train_test_split( lines, test_size=val_ratio, train_size = train_ratio ,random_state=42)
    
    train_file_name = train_file_name or os.path.join(os.path.dirname(data_file), "train.csv")
    val_file_name = val_file_name or os.path.join(os.path.dirname(data_file), "val.csv")

    
    with open(train_file_name, "w") as f:
        f.writelines(train)
    print("Wrote {} records to train".format(len(train)))
    
    with open(val_file_name, "w") as f:
        f.writelines(val)
    print("Wrote {} records to validation".format(len(val)))
    
    return train_file_name, val_file_name


In [ ]:
if prepare_dataset:
    from s3_util import S3Util
    
    s3util = S3Util()
    l_data_file = os.path.join(tmp, "dbpedia_csv", "train.csv")
    l_train, l_val = train_val_split(l_data_file)
    s3util.upload_file(l_train, s3_uri_train)
    s3util.upload_file(l_val, s3_uri_val)
    
    l_mini_train = os.path.join(os.path.dirname(l_data_file), "mini_train.csv")
    l_mini_val = os.path.join(os.path.dirname(l_data_file), "mini_val.csv") 
    l_train, l_val = train_val_split(l_data_file, l_mini_train, l_mini_val, val_ratio = 0.001, train_ratio=0.01)

    s3util.upload_file(l_mini_train, s3_uri_mini_train)
    s3util.upload_file(l_mini_val, s3_uri_mini_val)
    
    

In [ ]:
# Clean up temp file..
!rm -rf $tmp

## Train

This shows you how to train BERT on SageMaker using SPOT instances

In [ ]:
inputs_full =  {
    "train" : s3_uri_train,
    "val" : s3_uri_val,
    "class" : s3_uri_classes
}

inputs_sample =  {
    "train" : s3_uri_mini_train,
    "val" : s3_uri_mini_val,
    "class" : s3_uri_classes
}

# Using the full dataset can take a while 4-5 hours. So if you just quickly test the sample, use inputs_sample
inputs = inputs_sample

In [ ]:
sm_localcheckpoint_dir="/opt/ml/checkpoints/"

In [ ]:
instance_type = "ml.p3.8xlarge"
instance_type_gpu_map = {"ml.p3.8xlarge":4, "ml.p3.2xlarge": 1, "ml.p3.16xlarge":8}

In [ ]:
hp = {
"epochs" : 10,
"earlystoppingpatience" : 3,
# Increasing batch size might end up with CUDA OOM error, increase grad accumulation instead
"batch" : 8 * instance_type_gpu_map[instance_type],
"trainfile" :s3_uri_train.split("/")[-1],
"valfile" : s3_uri_val.split("/")[-1],
"classfile":s3_uri_classes.split("/")[-1],
# The number of steps to accumulate gradients for
"gradaccumulation" : 4,
"log-level":"INFO",
# This param depends on your model max pos embedding size or when large you might end up with CUDA OOM error    
"maxseqlen" : 512,
# Make sure the lr is quite small, as this is a pretrained model..
"lr":0.00001,
# Use finetuning (set to 1), if you only want to change the weights in the final classification layer.. 
"finetune": 0,
"checkpointdir" : sm_localcheckpoint_dir,
# Checkpoints once every n epochs
"checkpointfreq": 2
}



In [ ]:
hp

In [ ]:
inputs

In [ ]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainScore",
                     "Regex": "###score: train_score### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationScore",
                     "Regex": "###score: val_score### (\d*[.]?\d*)"}
                    ]

In [ ]:
# set True if you need spot instance
use_spot = True
train_max_run_secs =   2*24 * 60 * 60
spot_wait_sec =  5 * 60
max_wait_time_secs = train_max_run_secs +  spot_wait_sec

if not use_spot:
    max_wait_time_secs = None
    
# During local mode, no spot.., use smaller dataset
if instance_type == 'local':
    use_spot = False
    max_wait_time_secs = 0
    wait = True
    # Use smaller dataset to run locally
    inputs = inputs_sample


In [ ]:
job_type = "bert-classification"
base_name = "{}".format(job_type)

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     #entry_point='main_train_k_fold.py',
    entry_point='main.py',
                    source_dir = 'src',
                    role=role,
                    framework_version ="1.4.0",
                    py_version='py3',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters = hp,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    volume_size=30,
                    code_location=s3_code_path,
                    debugger_hook_config=False,
                    base_job_name =base_name,  
                    use_spot_instances = use_spot,
                    max_run =  train_max_run_secs,
                    max_wait = max_wait_time_secs,   
                    checkpoint_s3_uri=s3_checkpoint,
                    checkpoint_local_path=sm_localcheckpoint_dir)

estimator.fit(inputs, wait=True)

## Deploy BERT model

#### Inference container
Ideally the server containing should already have all the required dependencies installed to reduce start up time and ensure that the runtime enviornment is consistent. This can be implemented using a custom docker image.

But for this demo, to simplify, we will let the Pytorch container script model install the dependencies during start up. As a result, you will see some of the initial ping requests fail, until all dependencies are installed.


In [ ]:
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
role = get_execution_role()

model_uri = estimator.model_data

model = PyTorchModel(model_data=model_uri,
                     role=role,
                     py_version = "py3",
                     framework_version='1.4.0',
                     entry_point='serve.py',
                     source_dir='src')

predictor = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

### Invoke API

In [ ]:
data = ["Q-workshop is a Polish company located in Poznań that specializes in designand production of polyhedral dice",
        "ET is a sci-fi directed by steven spielberg"]

In [ ]:
import json


class TextSerDes:
    
     def serialize(self, x):
        data_bytes="\n".join(x).encode("utf-8")
        return data_bytes
    
     def deserialize(self, x, content_type):
        return json.loads(x.read().decode("utf-8")) 

In [ ]:

predictor.serializer = TextSerDes()
predictor.deserializer = TextSerDes()


response  = predictor.predict(data,  initial_args={ "Accept":"text/json", "ContentType" : "text/csv" }
                                   )

response 

## Delete endpoint

In [ ]:
predictor.delete_endpoint()